<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/NeuralNetworks/NeuronSimulation.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation of an artificial neuron

Deep neural networks consist of multiple layers, each made up of a collection of neurons. Every neuron calculates its activation—essentially its level of activity—based on the weighted connections it has with other neurons in the network. This activation reflects how the neuron responds to a given input.

The process each neuron follows is fairly simple. It starts by computing a weighted sum of the activations from all the neurons it's connected to—this is known as the net input. Then, it applies an activation function to this net input. There are several types of activation functions, and the one used affects the neuron's output. This resulting activation is then sent to the neurons in the next layer of the network.

In this notebook, you'll have the opportunity to explore how a single neuron works. You can experiment with different incoming activations, weights, and activation functions to see how they influence the neuron's output.

The following code was developed in collaboration with ChatGPT.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Dropdown, interact
from matplotlib.patches import Circle, FancyBboxPatch
from matplotlib import cm

# Activation functions and their names
def sigmoid(x): return 1 / (1 + np.exp(-x))
def tanh(x): return np.tanh(x)
def relu(x): return np.maximum(0, x)

activation_funcs = {
    "Sigmoid": (sigmoid, "Sigmoid Activation"),
    "Tanh": (tanh, "Tanh Activation"),
    "ReLU": (relu, "ReLU Activation")
}

# Helper to draw input arrows
def draw_arrow_to_neuron(ax, start, end, label, x_val, w_val):
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    ax.arrow(start[0], start[1], dx, dy, head_width=0.04, head_length=0.08, fc='black', ec='black')
    ax.text(start[0] - 0.07, start[1], f"{label}={x_val:.1f}", fontsize=9, va='center', ha='right')
    ax.text(start[0] + dx/2 - 0.4, start[1] + dy/2, f"w={w_val:.2f}", fontsize=10, color='blue')

# Main interactive neuron visualization
def draw_interactive_neuron(x1, x2, wbias, w1, w2, activation_name):
    x_bias = 1.0
    inputs = np.array([x1, x2, x_bias])
    weights = np.array([w1, w2, wbias])
    z = np.dot(inputs, weights)

    activation_fn, activation_title = activation_funcs[activation_name]
    output = activation_fn(z)

    # Normalize output for neuron size and color
    norm_output = (output + 1) / 2 if activation_name == "Tanh" else output
    norm_output = np.clip(norm_output, 0, 1)
    color = cm.viridis(norm_output)
    radius = 0.25 + 0.25 * norm_output

    # Set up full figure with activation subplot
    fig = plt.figure(figsize=(8, 5))
    grid = fig.add_gridspec(1, 2, width_ratios=[2.5, 1])

    ax = fig.add_subplot(grid[0])
    ax.set_xlim(-1.8, 3)
    ax.set_ylim(-1.5, 1.5)
    ax.axis('off')

    neuron_center = (1.6, 0)
    net_input_box_center = (0.3, 0)

    # Draw arrows from inputs
    input_coords = [(-1.3, 1), (-1.3, -1), (-1.3, 0)]
    input_labels = ['x1', 'x2', 'bias']
    input_values = [x1, x2, 1.0]
    weight_values = [w1, w2, wbias]

    for start, label, x_val, w_val in zip(input_coords, input_labels, input_values, weight_values):
        draw_arrow_to_neuron(ax, start, net_input_box_center, label, x_val, w_val)

    # Net input box
    box_width, box_height = 0.7, 0.45
    box = FancyBboxPatch((net_input_box_center[0] - box_width / 2, net_input_box_center[1] - box_height / 2),
                         width=box_width, height=box_height,
                         boxstyle="round,pad=0.05", linewidth=1, edgecolor='black', facecolor='lightblue')
    ax.add_patch(box)
    ax.text(*net_input_box_center, f"z = {z:.2f}", ha='center', va='center', fontsize=10, weight='bold')

    # Arrow from box to neuron
    ax.arrow(net_input_box_center[0] + box_width / 2, net_input_box_center[1],
             neuron_center[0] - net_input_box_center[0] - box_width / 2, 0,
             head_width=0.04, head_length=0.08, fc='black', ec='black')

    # Neuron circle
    neuron = Circle(neuron_center, radius=radius, color=color, ec='black')
    ax.add_patch(neuron)
    ax.text(*neuron_center, f"{output:.2f}", ha='center', va='center', fontsize=10, weight='bold', color='white')

    # Output arrow
    ax.arrow(neuron_center[0] + radius, neuron_center[1], 0.6, 0,
             head_width=0.04, head_length=0.08, fc='green', ec='green')
    ax.text(neuron_center[0] + radius + 0.7, 0.1, "Output", fontsize=9, color='green')

    # Draw activation function graph
    ax_act = fig.add_subplot(grid[1])
    x_vals = np.linspace(-6, 6, 200)
    y_vals = activation_fn(x_vals)
    ax_act.plot(x_vals, y_vals, label=activation_name)
    ax_act.axvline(z, color='red', linestyle='--', linewidth=1)
    ax_act.plot(z, output, 'ro', label='Current Output')
    ax_act.set_title(activation_title, fontsize=11)
    ax_act.set_xlabel("z")
    ax_act.set_ylabel("activation(z)")
    ax_act.grid(True)

    # Adjust y-axis limits by activation type
    if activation_name == "Tanh":
        ax_act.set_ylim(-1.1, 1.1)
    else:
        ax_act.set_ylim(-0.1, 1.1)

    ax_act.legend(fontsize=8)

    plt.suptitle(f"Activation: {activation_name} | Net input z: {z:.2f} | Output: {output:.3f}", fontsize=12)
    plt.tight_layout()
    plt.show()

# Interactive UI
interact(
    draw_interactive_neuron,
    x1=FloatSlider(value=1.0, min=-5, max=5, step=0.1, description="x1"),
    x2=FloatSlider(value=1.0, min=-5, max=5, step=0.1, description="x2"),
    wbias=FloatSlider(value=0.0, min=-5, max=5, step=0.1, description="w_bias"),
    w1=FloatSlider(value=0.5, min=-5, max=5, step=0.1, description="w1"),
    w2=FloatSlider(value=0.5, min=-5, max=5, step=0.1, description="w2"),
    activation_name=Dropdown(options=["Sigmoid", "Tanh", "ReLU"], value="Sigmoid", description="Activation")
)


interactive(children=(FloatSlider(value=1.0, description='x1', max=5.0, min=-5.0), FloatSlider(value=1.0, desc…

<function __main__.draw_interactive_neuron(x1, x2, wbias, w1, w2, activation_name)>